# Create `no_snow_ice.nc` File for NEMO Surface Forcing

Create a netCDF4 file containing 2 variables named `snow` and `ice`.
The coordinates of the variable are `y` and `x`.
The values of `snow` and `ice` at all points in the domain is floating point zero.

The resulting `no_snow_ice.nc` file can be used as an annual climatology in NEMO atmospheric forcing
that does not require on-the-fly interpolation.
It imposes a no snow, ever condition on the NEMO configuration.
The no ice, ever condition that it provides works in conjunction with the code in `sbcice_if.F90` contributed by Michael Dunphy
to provide a minimal ice-model substitute. 
In Michael's words,
> "The point is to make sure water temperatures don’t go below the local freezing point, 
> and there are some limits on heat exchanges as well."

In [1]:
import datetime
from pathlib import Path

import numpy
import xarray

Parameters:

In [2]:
domain_sizes = {
    # "grid name": (y, x)
    "SalishSeaCast-202108": (898, 398),
    "SalishSeaCast-202405": (908, 398),
    "SalishSeaCast_double-202405": (1816, 796),
    "sss150-20210709": (826, 710),
}
domain_size = domain_sizes["sss150-20210709"]

dataset_titles = {
    "SalishSeaCast": "SalishSeaCast NEMO Domain Yearly Snowfall and Sea Ice Coverage Climatology",
    "sss150": "sss150 NEMO Domain Yearly Snowfall and Sea Ice Coverage Climatology",
}
dataset_title = dataset_titles["sss150"]

notebook_url = "https://github.com/SalishSeaCast/analysis-doug/blob/main/notebooks/sss150/NoSnow.ipynb"

# isolate the probable user-specific part of repo paths to make the notebook portable
repos_prefix = Path("/data/dlatorne/MEOPAR")
grid_paths = {
    "SalishSeaCast": Path("grid"),
    "sss150": Path("grid", "sss150"),
}
grid_path = grid_paths["sss150"]
no_snow_ice_paths = {
    "SalishSeaCast-202108": repos_prefix / grid_path / "no_snow_ice_202108.nc",
    "SalishSeaCast-202405": repos_prefix / grid_path / "no_snow_ice_202405.nc",
    "SalishSeaCast_double-202405": repos_prefix / grid_path / "no_snow_ice_double_202405.nc",
    "sss150-20210709": repos_prefix / grid_path / "no_snow_ice.nc",
}
no_snow_ice_path = no_snow_ice_paths["sss150-20210709"]

domain_size, no_snow_ice_path, dataset_title

((826, 710),
 PosixPath('/data/dlatorne/MEOPAR/grid/sss150/no_snow_ice.nc'),
 'sss150 NEMO Domain Yearly Snowfall and Sea Ice Coverage Climatology')

Coordinates:

In [3]:
y_indices = numpy.arange(domain_size[0], dtype=numpy.int64)
y_coord = xarray.DataArray(
    name="y",
    data=y_indices,
    coords={"y": y_indices},
    attrs={
        "long_name": "Grid Y",
        "standard_name": "y",
        "units": "count",
        "comment": "y values are grid indices in the model y-direction",
    },
)

x_indices = numpy.arange(domain_size[1], dtype=numpy.int64)
x_coord = xarray.DataArray(
    name="x",
    data=x_indices,
    coords={"x": x_indices},
    attrs={
        "long_name": "Grid X",
        "standard_name": "x",
        "units": "count",
        "comment": "x values are grid indices in the model x-direction",
    },
)

`snow` and `ice` variables:

In [4]:
zeros = numpy.zeros(domain_size, dtype=numpy.float32)

snow_array = xarray.DataArray(
    name="snow",
    data=zeros,
    coords={"y": y_coord, "x": x_coord},
    attrs={
        "long_name": "Snow Precipitation Rate",
        "standard_name": "snowfall_flux",
        "units": "kg m-2 s-1",
        
    }
)
ice_array = xarray.DataArray(
    name="ice",
    data=zeros,
    coords={"y": y_coord, "x": x_coord},
    attrs={
        "long_name": "Sea Ice Coverage Fraction",
        "standard_name": "sea_ice_area_fraction",
        "units": "1",
        
    }
)


In [5]:
snow_array

<xarray.DataArray 'snow' (y: 826, x: 710)> Size: 2MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * y        (y) int64 7kB 0 1 2 3 4 5 6 7 8 ... 818 819 820 821 822 823 824 825
  * x        (x) int64 6kB 0 1 2 3 4 5 6 7 8 ... 702 703 704 705 706 707 708 709
Attributes:
    long_name:      Snow Precipitation Rate
    standard_name:  snowfall_flux
    units:          kg m-2 s-1

In [6]:
ice_array

<xarray.DataArray 'ice' (y: 826, x: 710)> Size: 2MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * y        (y) int64 7kB 0 1 2 3 4 5 6 7 8 ... 818 819 820 821 822 823 824 825
  * x        (x) int64 6kB 0 1 2 3 4 5 6 7 8 ... 702 703 704 705 706 707 708 709
Attributes:
    long_name:      Sea Ice Coverage Fraction
    standard_name:  sea_ice_area_fraction
    units:          1

Dataset:

In [7]:
no_snow_ice_ds = xarray.Dataset(
    data_vars={
        "snow": snow_array,
        "ice": ice_array,
    },
    coords={"y": y_coord, "x": x_coord},
    attrs={
        "conventions": "CF-1.6",
        "title": dataset_title,
        "institution": "Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia",
        "source": notebook_url,
        "comment": "Annual climatology HRDPS-like atmospheric forcing file for NEMO that always supplies zero as the solid precipitation rate and sea ice coverage fraction values.",
        "history": f"{datetime.datetime.now():%Y-%m-%d %H:%M}: Created.",
    },
)

no_snow_ice_ds

<xarray.Dataset> Size: 5MB
Dimensions:  (y: 826, x: 710)
Coordinates:
  * y        (y) int64 7kB 0 1 2 3 4 5 6 7 8 ... 818 819 820 821 822 823 824 825
  * x        (x) int64 6kB 0 1 2 3 4 5 6 7 8 ... 702 703 704 705 706 707 708 709
Data variables:
    snow     (y, x) float32 2MB 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ice      (y, x) float32 2MB 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    conventions:  CF-1.6
    title:        sss150 NEMO Domain Yearly Snowfall and Sea Ice Coverage Cli...
    institution:  Dept of Earth, Ocean & Atmospheric Sciences, University of ...
    source:       https://github.com/SalishSeaCast/analysis-doug/blob/main/no...
    comment:      Annual climatology HRDPS-like atmospheric forcing file for ...
    history:      2024-10-17 11:53: Created.

Encoding for netCDF4 storage:

In [8]:
encoding = {
    var: {
        "chunksizes": no_snow_ice_ds[var].shape,
        "zlib": True,
        "complevel": 4,
        "shuffle": True,
    }
    for var in ("y", "x", "snow", "ice")
}
for var in ("snow", "ice"):
    encoding[var]["_FillValue"] = 0,
    
encoding

{'y': {'chunksizes': (826,), 'zlib': True, 'complevel': 4, 'shuffle': True},
 'x': {'chunksizes': (710,), 'zlib': True, 'complevel': 4, 'shuffle': True},
 'snow': {'chunksizes': (826, 710),
  'zlib': True,
  'complevel': 4,
  'shuffle': True,
  '_FillValue': (0,)},
 'ice': {'chunksizes': (826, 710),
  'zlib': True,
  'complevel': 4,
  'shuffle': True,
  '_FillValue': (0,)}}

Store netCDF file:

In [9]:
no_snow_ice_ds.to_netcdf(no_snow_ice_path, encoding=encoding, engine="h5netcdf")